# Building A Runtime Program

In [3]:
import sys
import json

from qiskit.providers.ibmq.runtime import UserMessenger, ProgramBackend

## Skeleten of a runtime program

In [6]:
def program(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    """Function that does classical-quantum calculation."""
    # UserMessenger can be used to publish interim results.
    user_messenger.publish("This is an interim result.")
    return "final result"

def main(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    """This is the main entry point of a runtime program.

    The name of this method must not change. It also must have ``backend``
    and ``user_messenger`` as the first two positional arguments.

    Args:
        backend: Backend for the circuits to run on.
        user_messenger: Used to communicate with the program user.
        kwargs: User inputs.
    """
    # Massage the input if necessary.
    result = program(backend, user_messenger, **kwargs)
    # Final result can be directly returned
    return result

## A simple runtime program

In [7]:
"""A sample runtime program that submits random circuits for user-specified iterations."""
import random
from qiskit import transpile
from qiskit.circuit.random import random_circuit



def prepare_circuits(backend):
    """Generate a random circuit.

    Args:
        backend: Backend used for transpilation.

    Returns:
        Generated circuit.
    """
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                             seed=random.randint(0, 1000))
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop('iterations', 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "All done!"

### Test the program

In [8]:
from qiskit import Aer
from qiskit.providers.ibmq.runtime import UserMessenger

backend = Aer.get_backend('qasm_simulator')
user_messenger = UserMessenger()

In [9]:
inputs = {"iterations": 3}

main(backend, user_messenger, **inputs)

{"iteration": 0, "counts": {"00110": 4, "01101": 1020}}
{"iteration": 1, "counts": {"00000": 1024}}
{"iteration": 2, "counts": {"01000": 19, "01100": 1005}}


'All done!'

### Uploading the program

In [10]:
program_details = {
  "name": "sample-program_John_Stenger",
  "description": "A sample runtime program.",
  "max_execution_time": 300,
  "version": "1.0",
  "backend_requirements": {"min_num_qubits":  5},
  "parameters": [
    {"name": "iterations", "description": "Number of iterations to run. Each iteration generates and runs a random circuit.", "type": "int", "required": True}
  ],
  "return_values": [
    {"name": "-", "description": "A string that says 'All done!'.", "type": "string"}
  ],
  "interim_results": [
    {"name": "iteration", "description": "Iteration number.", "type": "int"},
    {"name": "counts", "description": "Histogram data of the circuit result.", "type": "dict"}
  ]
}

out_file = open("runtime_test_details.json", "w")
json.dump(program_details, out_file)
out_file.close()

in_file = open("runtime_test_details.json",)
data = json.load(in_file)
in_file.close()

print(data)

{'name': 'sample-program_John_Stenger', 'description': 'A sample runtime program.', 'max_execution_time': 300, 'version': '1.0', 'backend_requirements': {'min_num_qubits': 5}, 'parameters': [{'name': 'iterations', 'description': 'Number of iterations to run. Each iteration generates and runs a random circuit.', 'type': 'int', 'required': True}], 'return_values': [{'name': '-', 'description': "A string that says 'All done!'.", 'type': 'string'}], 'interim_results': [{'name': 'iteration', 'description': 'Iteration number.', 'type': 'int'}, {'name': 'counts', 'description': 'Histogram data of the circuit result.', 'type': 'dict'}]}


In [11]:
%%writefile runtime_test.py

"""A sample runtime program that submits random circuits for user-specified iterations."""
import random
from qiskit import transpile
from qiskit.circuit.random import random_circuit



def prepare_circuits(backend):
    """Generate a random circuit.

    Args:
        backend: Backend used for transpilation.

    Returns:
        Generated circuit.
    """
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                             seed=random.randint(0, 1000))
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop('iterations', 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "All done!"

Overwriting runtime_test.py


In [5]:
import os
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')  # Substitute with your provider.


In [13]:
sample_program_json = os.path.join(os.getcwd(), "runtime_test_details.json")
sample_program_data = os.path.join(os.getcwd(), "runtime_test.py")

In [10]:
# This will fail if a sample-program already exists.
program_id = provider.runtime.upload_program(
    data=sample_program_data,
    metadata=sample_program_json
)
print(program_id)

sample-program-john-stenger


## Using and deleting the program

In [1]:
program_id = 'sample-program-john-stenger'

In [6]:
my_program = provider.runtime.program(program_id)

print(my_program)

sample-program-john-stenger:
  Name: sample-program_John_Stenger
  Description: A sample runtime program.
  Version: 1.0
  Creation date: 2021-10-26T17:56:38.000000
  Max execution time: 300
  Input parameters:
    - iterations:
      Description: Number of iterations to run. Each iteration generates and runs a random circuit.
      Type: int
      Required: True
  Interim results:
    - iteration:
      Description: Iteration number.
      Type: int
    - counts:
      Description: Histogram data of the circuit result.
      Type: dict
  Returns:
    - -:
      Description: A string that says 'All done!'.
      Type: string


### Printing only final results

In [9]:
#backend = provider.backend.ibmq_qasm_simulator
#backend = provider.get_backend('ibmq_bogota')
#backend = provider.get_backend('ibmq_manila')
backend = provider.get_backend('ibmq_belem')
options = {'backend_name': backend.name()}
inputs = {"iterations": 3}
job = provider.runtime.run(program_id, options=options, inputs=inputs)

In [10]:
job.result()

'All done!'

### Print intermediate results

In [14]:
interm_results = []
def my_callback(job_id, data):
    interm_results.append(data)

In [15]:
job2 = provider.runtime.run(program_id, options=options, inputs=inputs, callback=my_callback)

In [16]:
job2.result()

'All done!'

In [17]:
interm_results

[{'iteration': 0,
  'counts': {'00110': 6, '00010': 8, '10110': 502, '10010': 508}},
 {'iteration': 1,
  'counts': {'11001': 2,
   '10101': 2,
   '01001': 1,
   '00100': 7,
   '11000': 17,
   '11101': 43,
   '00001': 18,
   '10001': 43,
   '00000': 113,
   '01101': 22,
   '01100': 124,
   '01000': 4,
   '10100': 23,
   '11100': 303,
   '10000': 302}},
 {'iteration': 2, 'counts': {'00001': 491, '00000': 533}}]

### Deleting the program

In [71]:
#Delete the program
provider.runtime.delete_program(program_id)